# Customer Segmentation and Classification
Problem:
User clickstream data and information about a group of hotels is provided.

Objective:
Segment users into 3 clusters and predict to which cluster a new customer belongs

In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Tell iPython to include plots inline in the notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Read dataset
globalData = pd.read_csv("globalTrain.csv")

#print globalData.head() # print the first 5 rows
globalData.head()

In [ ]:
#Checking variables data types
globalData.dtypes

In [ ]:
#Renaming columns and changing data types

globalData=globalData.rename(columns = {' HotelCode':'HotelCode'})
globalData=globalData.rename(columns = {' Age':'Age'})
globalData=globalData.rename(columns = {' Gender':'Gender'})
globalData=globalData.rename(columns = {' Number of Rooms':'Number of Rooms'})
globalData=globalData.rename(columns = {' Check in date':'Check in date'})
globalData=globalData.rename(columns = {' Check Out Date':'Check Out Date'})
globalData=globalData.rename(columns = {' Seen Price':'Seen Price'})
globalData=globalData.rename(columns = {' isClicked':'isClicked'})
globalData=globalData.rename(columns = {' isBooked':'isBooked'})
globalData=globalData.rename(columns = {' Segment':'Segment'})

globalData['Booking Date'] =  pd.to_datetime(globalData['Booking Date'])
globalData['Check in date'] =  pd.to_datetime(globalData['Check in date'])
globalData['Check Out Date'] =  pd.to_datetime(globalData['Check Out Date'])
globalData['isClicked'] =  globalData['isClicked'].astype(str)
globalData['isBooked'] =  globalData['isBooked'].astype(str)

globalData['Stay Period'] = (globalData['Check Out Date'] - globalData['Check in date'])/np.timedelta64(1, 'D');
globalData['Travel Gap'] = (globalData['Check in date'] - globalData['Booking Date'])/np.timedelta64(1, 'D');

In [ ]:
globalData.head()

In [ ]:
globalData.describe()

In [ ]:
#Now separate training data
X_train=globalData.iloc[:,[3,4,5,8,9,10,16,17,18,19]];
#X_train=globalData.iloc[:,[3]];
X_train['Gender'] = X_train['Gender'].replace(['male', 'female'], [1, 0])
X_train['isClicked'] = X_train['isClicked'].replace(['True', 'False'], [1, 0])
X_train['isBooked'] = X_train['isBooked'].replace(['True', 'False'], [1, 0])
X_train[X_train < 0] = 0

In [ ]:
# Import clustering modules
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=10)
pca.fit(X_train)
# Print the components and the amount of variance in the data contained in each dimension
#print pca.components_

In [ ]:
# TODO: First we reduce the data to two dimensions using PCA to capture variation
pca = PCA(n_components=3)
reduced_data = pca.fit_transform(X_train)
print(reduced_data[:10]) #print upto 10 elements
print(pca.components_)
print(pca.explained_variance_ratio_)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [ ]:
#Train the clustering algorythm
kmeans = KMeans(init='k-means++', n_clusters=3, n_init=10)
kmeans.fit(reduced_data)
y_kmeans = kmeans.predict(reduced_data)

In [ ]:
plt.scatter(reduced_data[:, 0], reduced_data[:, 2], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 2], c='black', s=200, alpha=0.5);

In [ ]:
globalData['cluster'] = y_kmeans

In [ ]:
# Comparing Clusters
import seaborn as sns
 
sns.boxplot(x=globalData["cluster"], y=globalData["Seen Price"] )
plt.show()

In [ ]:
globalData.columns

In [ ]:
#Preprocessing for scaling
X =globalData.iloc[:,[3,4,5,8,9,10,16,17,18,19]];

X['Gender'] = X['Gender'].replace(['male', 'female'], [1, 0])
X['isClicked'] = X['isClicked'].replace(['True', 'False'], [1, 0])
X['isBooked'] = X['isBooked'].replace(['True', 'False'], [1, 0])

X.head()

In [ ]:
#Scaling values to 0-100

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(X))

scaled_X = scaler.transform(X)
X1 = pd.DataFrame(scaled_X)
X1.columns = X.columns

X1.head()
X2 = X1*100
X2['cluster'] = y_kmeans
X2.describe()

In [ ]:
#Groupng data into clusters

grouped = X2.groupby('cluster').agg({'Age': ['median'], 'Seen Price': ['median'], 'Star Rating': ['median'], 'TripAdvisor Rating': ['median'], 'Stay Peri

In [ ]:
grouped.reset_index()
grouped.columns = ['Age', 'Seen Price', 'Star Rating', 'TripAdvisor Rating', 'Stay Period']
grouped.head()

In [ ]:
#Creating radar charts for clusters


# Libraries
import matplotlib.pyplot as plt
import pandas as pd
from math import pi

# Set data
df = pd.DataFrame({
'group': ['0','1','2'],
'Age': grouped['Age'].tolist(),
'Seen Price': grouped['Seen Price'].tolist(),
'Star Rating': grouped['Star Rating'].tolist(),
'TripAdvisor Rating': grouped['TripAdvisor Rating'].tolist(),
'Stay Period': grouped['Stay Period'].tolist(),
})

# ------- PART 1: Define a function that do a plot for one line of the dataset!

def make_spider( row, title, color):

# number of variable
    categories=list(df)[1:]
    N = len(categories)

# What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

# Initialise the spider plot
    ax = plt.subplot(2,2,row+1, polar=True, )

# If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

# Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=8)

# Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([25,50,75], ["25","50","75"], color="grey", size=7)
    plt.ylim(0,100)

# Ind1
    values=df.loc[row].drop('group').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

# Add a title
    plt.title(title, size=11, color=color, y=1.1)

# ------- PART 2: Apply to all individuals
# initialize the figure
my_dpi=96
plt.figure(figsize=(1000/my_dpi, 1000/my_dpi), dpi=my_dpi)

# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))

# Loop to plot
for row in range(0, len(df.index)):
    make_spider( row=row, title='group '+df['group'][row], color=my_palette(row))

In [ ]:

print(len(globalData))
globalData.head()

In [ ]:
#Now separate training data - Target
#X_train=globalData.drop(['Segment', 'cluster'], axis=1)
y_train=globalData.iloc[:,[20]]

print(X_train.head())
print(y_train.head())

In [ ]:
from sklearn.model_selection import train_test_split
# First, decide how many training vs test samples you want
num_test = 40000
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size=num_test)
print("Training set: {} samples".format(X_train_cv.shape[0]))
print("Test set: {} samples".format(X_test_cv.shape[0]))

In [ ]:
import time
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print("Done!\nTraining time (secs): {:.3f}".format(end - start))
    return (end - start)

In [ ]:
# Fit model to training data
#Best classifier
import time as time
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm, naive_bayes
from sklearn import linear_model
from  sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import PassiveAggressiveClassifier

#clf = DecisionTreeClassifier(random_state=0)
#clf = DecisionTreeClassifier(min_samples_split= 2, max_leaf_nodes= 32, criterion= 'entropy',
#                             max_depth = 9, min_samples_leaf=179)
#clf = DecisionTreeClassifier(min_samples_split= 21, max_leaf_nodes= 20, criterion= 'gini',
#                             max_depth = None, min_samples_leaf=11)

#dtr_params = {'criterion':("gini","entropy")}
#dtc2 = DecisionTreeClassifier(random_state=0)
#clf = linear_model.LogisticRegression()
#clf = grid_search.GridSearchCV(dtc2, dtr_params)

#dtr_params = {'criterion':("gini","entropy"),'presort':("True","False"),
#              'min_weight_fraction_leaf':(0,0.25,0.5), 'min_samples_leaf':(1,2,3),
#              'min_samples_split':(2,4,8,16,32),'min_samples_split':(2,4,8,16), 
#              'max_features':("auto","sqrt","log2"),'max_depth':np.arange(1,5,1)}
#dtc2 = DecisionTreeClassifier(random_state=0)
#clf = grid_search.GridSearchCV(dtc2, dtr_params)   

#from  sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier()
#clf1 = RandomForestClassifier(n_estimators=20)
# use a full grid over all parameters
#param_grid = {#"max_depth": [3, None],
              #"max_features": [1, 3, 10],
              #"min_samples_split": [1, 3, 10],
              #"min_samples_leaf": [1, 3, 10],
              #"bootstrap": [True, False],
              # "criterion": ["gini", "entropy"]
              #  "n_estimators": [5, 20,30]}
#clf = GridSearchCV(clf1, param_grid=param_grid)

#parameters={'C' : [.005,.05,.5,1.,10.,100.,],
#'fit_intercept' : [True, False],
#'class_weight': [ None,'balanced'],
#'random_state' : [None,42],
#'penalty': ['l1', 'l2']
#}
#clf = svm.SVC()

#SVC does not work
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 20]}
#svr = svm.SVC()
#clf = SVC(kernel="linear", C=1.0)

#Bad results
#clf= GaussianNB()

#clf=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
#    max_iter=-1, probability=False, random_state=None, shrinking=True,
#    tol=0.001, verbose=False)

#from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression()


#from sklearn.linear_model import SGDRegressor
#clf = SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
#             fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
#             loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
#             random_state=None, shuffle=True, verbose=0, warm_start=False)

#from sklearn.naive_bayes import BernoulliNB
#clf = BernoulliNB()

#from sklearn.linear_model import PassiveAggressiveClassifier
clf=PassiveAggressiveClassifier()

#Works
#from sklearn.neighbors import KNeighborsClassifier
#clf = KNeighborsClassifier(n_neighbors=35)

#Works
#from  sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(criterion='entropy',n_estimators=100, max_features=None,random_state=None)
#clf = RandomForestClassifier(n_estimators=20,min_samples_split=4)


#Works
#from  sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators=100)
#clf = RandomForestClassifier(n_estimators=100,max_features=None)

#Works
#from  sklearn.ensemble import AdaBoostClassifier
#clf = AdaBoostClassifier(n_estimators=50)
#clf = AdaBoostClassifier(n_estimators=60,learning_rate=0.65)

#from sklearn.tree import DecisionTreeClassifier
#clf2 = DecisionTreeClassifier(min_samples_split= 2, max_leaf_nodes= 20, criterion= 'gini',
#                             max_depth = None, min_samples_leaf=1)
#clf = AdaBoostClassifier(base_estimator=clf2,n_estimators=1,learning_rate=18,algorithm='SAMME')


#clf1 = AdaBoostClassifier()
#param_grid = { "n_estimators": [5, 20,30]}
#lf = GridSearchCV(clf1, param_grid=param_grid)

#from  sklearn.ensemble import RandomForestClassifier
#clf1 = RandomForestClassifier(n_estimators=20)
#clf = AdaBoostClassifier(base_estimator=clf1,n_estimators=50)

#Works
#from  sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#clf = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto', tol=0.0001)

#Works
#from  sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0, store_covariances=False, tol=0.000000001)

train_classifier(clf, X_train_cv, y_train_cv) # note: using entire training set here
#print clf # you can inspect the learned model by printing it

print("Successfull!!")

In [ ]:
#Predict on training set and compute F1 score
from sklearn.metrics import f1_score
def predict_labels(clf, features, target):
    #print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    #print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    #return f1_score(target.values, y_pred, pos_label='yes')
    return f1_score(target.values, y_pred, average='micro') , (end - start)

In [ ]:
#print y_train.head()
#print X_train.head()
train_cv_f1_score = predict_labels(clf, X_train_cv, y_train_cv)
#print "F1 score for training set: {}".format(train_cv_f1_score)
# Predict on test data
test_cv_f1_score = predict_labels(clf, X_test_cv, y_test_cv)
#print "F1 score for test set: {}".format(test_cv_f1_score)

In [ ]:
#Report
from astropy.table import Table, Column
import time as time
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm, naive_bayes
#from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import linear_model
from  sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.ensemble import AdaBoostClassifier
from  sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
noOfTypes =10;
t1 = Table(names=('Type','Training time' ,'Prediction time-Training', 'F1 score-Training', 
                  'Prediction time-Testing', 'F1 score-Testing'), dtype=('S25', 'f4','f4', 'f4','f4', 'f4'))

for i in range(0,noOfTypes):
    if i==0:
        clf=PassiveAggressiveClassifier();
    if i==1:
        clf=GaussianNB(); 
    if i==2:    
        clf = DecisionTreeClassifier(random_state=0)
    if i==3:    
        clf = RandomForestClassifier()
    if i==4:    
        clf = BernoulliNB()
    if i==5:    
        clf = LogisticRegression()
    if i==6:    
        clf = KNeighborsClassifier()
    if i==7:    
        clf = AdaBoostClassifier()
    if i==8:    
        clf = LinearDiscriminantAnalysis()
    if i==9:    
        clf = QuadraticDiscriminantAnalysis()
        
    clfType=clf.__class__.__name__;
    trainTime=train_classifier(clf, X_train_cv, y_train_cv);
    retVec=predict_labels(clf, X_train_cv, y_train_cv)
    trainF1Score=retVec[0];
    trainTime=retVec[1];
    retVec1=predict_labels(clf, X_test_cv, y_test_cv)
    testF1Score=retVec1[0];
    testTime=retVec1[1];
    t1.add_row((clfType,trainTime,trainTime, trainF1Score,testTime, testF1Score))

#t1 = Table(names=('Type','Training time' ,'Prediction time-Training', 'F1 score-Training', 'Prediction time-Testing', 'F1 score-Testing'), dtype=('S13', 'f4','f4', 'f4','f4', 'f4'))
#t1.add_row(('SVC',1,100, 3,4, 5))
#t1.add_row(('SVC',2, 200, 33,44, 55))

t1    